# Evaluation
Calculating metrics for the evaluation of the trained models

In [391]:
# imports
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np
from scipy.special import kl_div

In [392]:
# variables
data_input_filepath = './pickles/reservations_testing.pickle'
training_data_input_filepath = './pickles/reservations_training.pickle'

large_grid_poisson_demand_input_filepath = './pickles/demand/large_grid_poisson_demand_prediction.pickle'
large_grid_negative_binomial_demand_input_filepath = './pickles/demand/large_grid_negative_binomial_demand_prediction.pickle'
small_grid_poisson_demand_input_filepath = './pickles/demand/small_grid_poisson_demand_prediction.pickle'
small_grid_negative_binomial_demand_input_filepath = './pickles/demand/small_grid_negative_binomial_demand_prediction.pickle'
voronoi_grid_poisson_demand_input_filepath = './pickles/demand/voronoi_grid_poisson_demand_prediction.pickle'
voronoi_grid_negative_binomial_demand_input_filepath = './pickles/demand/voronoi_grid_negative_binomial_demand_prediction.pickle'

ml_evaluation_output_path = './pickles/results/ml_evaluation_results.pickle'
dl_evaluation_output_path = './pickles/results/dl_evaluation_results.pickle'
evaluation_output_path = './pickles/results/combined_evaluation_results.pickle'

In [393]:
# load data
large_grid_poisson_demand = pd.read_pickle(large_grid_poisson_demand_input_filepath)
large_grid_poisson_demand.set_index('timestamp', inplace=True)

large_grid_negative_binomial_demand = pd.read_pickle(large_grid_negative_binomial_demand_input_filepath)
large_grid_negative_binomial_demand.set_index('timestamp', inplace=True)

small_grid_poisson_demand = pd.read_pickle(small_grid_poisson_demand_input_filepath)
small_grid_poisson_demand.set_index('timestamp', inplace=True)

small_grid_negative_binomial_demand = pd.read_pickle(small_grid_negative_binomial_demand_input_filepath)
small_grid_negative_binomial_demand.set_index('timestamp', inplace=True)

voronoi_grid_poisson_demand = pd.read_pickle(voronoi_grid_poisson_demand_input_filepath)
voronoi_grid_poisson_demand.set_index('timestamp', inplace=True)

voronoi_grid_negative_binomial_demand = pd.read_pickle(voronoi_grid_negative_binomial_demand_input_filepath)
voronoi_grid_negative_binomial_demand.set_index('timestamp', inplace=True)

In [394]:
large_grid_poisson_demand.head()

,195,266,212,163,126,111,127,109,161,162,...,199,137,234,124,333,299,132,69,182,12
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.467590,0.120662,0.233912,0.148309,0.877903,0.011163,0.567526,0.169160,1.087699,0.364779,...,1.203686e-04,3.222315e-06,5.551061e-06,1.235775e-10,1.247627e-06,5.315257e-09,9.895447e-08,6.400777e-07,6.029670e-08,2.279266e-10
2017-10-14 15:00:00,0.595860,0.182780,0.367664,0.241795,1.281619,0.018331,0.840228,0.250466,1.196791,0.441283,...,2.369359e-04,1.474516e-04,2.748215e-06,3.437231e-08,6.899338e-08,5.095876e-09,5.665706e-07,2.243172e-06,3.169538e-06,2.382457e-08
2017-10-14 18:00:00,0.366629,0.096230,0.204596,0.131406,0.721692,0.012694,0.481221,0.127169,0.661329,0.234874,...,2.451119e-04,1.683356e-04,2.730139e-06,1.192115e-08,7.089240e-08,1.063116e-08,1.933066e-07,1.683959e-06,6.087815e-08,2.279218e-10
2017-10-14 21:00:00,0.108745,0.030246,0.057757,0.040290,0.246705,0.002134,0.130581,0.030229,0.307402,0.089594,...,1.147403e-06,3.834831e-06,3.651018e-06,1.743936e-10,8.108581e-08,3.340637e-11,9.138022e-10,2.011115e-09,7.829689e-08,2.841300e-10
2017-10-15 06:00:00,0.153784,0.055332,0.153811,0.051593,0.570707,0.014458,0.314064,0.099979,0.140491,0.140167,...,9.388826e-07,5.274224e-08,5.854028e-07,1.292106e-10,6.870069e-07,2.491744e-11,8.233381e-07,1.310713e-13,3.646617e-06,2.357985e-10


In [395]:
# load reservation data
validation_data = pd.read_pickle(data_input_filepath)
validation_data['startTime'] = pd.to_datetime(validation_data['startTime'])
validation_data.set_index('startTime', inplace=True)
validation_data.drop(columns=['endTime', 'temperature', 'precipitation', 'startLat', 'startLon', 'endLat', 'endLon'], inplace=True)
validation_data.head()

,voronoi_grid_id,small_grid_id,large_grid_id,community_small_grid_id,community_voronoi_grid_id
startTime,,,,,
2018-04-30 10:43:49,448,5059,195,13,0
2022-09-16 15:46:43,339,3271,127,7,0
2019-09-07 20:00:38,949,6435,267,8,7
2022-06-24 11:28:29,454,4972,195,7,7
2018-07-16 15:15:15,155,4295,179,7,4


In [396]:
large_grid_validation_data = validation_data.resample('3H').large_grid_id.value_counts().unstack().fillna(0)
large_grid_validation_data['hour'] = large_grid_validation_data.index.hour
large_grid_validation_data['day_of_week'] = large_grid_validation_data.index.dayofweek
large_grid_validation_data['month'] = large_grid_validation_data.index.month
large_grid_validation_data = large_grid_validation_data[
    (large_grid_validation_data['month'] != 1) &
    (large_grid_validation_data['month'] != 2) &
    (large_grid_validation_data['month'] != 11) &
    (large_grid_validation_data['month'] != 12)
    ]
large_grid_validation_data.drop(columns=['hour', 'day_of_week', 'month'], inplace=True)
large_grid_validation_data.head()

large_grid_id,12,21,53,69,70,76,90,103,107,108,...,281,282,283,284,297,298,299,300,301,313
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [397]:
# transform data
small_grid_validation_data = validation_data.resample('3H').small_grid_id.value_counts().unstack().fillna(0)
small_grid_validation_data['hour'] = small_grid_validation_data.index.hour
small_grid_validation_data['day_of_week'] = small_grid_validation_data.index.dayofweek
small_grid_validation_data['month'] = small_grid_validation_data.index.month
small_grid_validation_data.sort_index(inplace=True)
small_grid_validation_data = small_grid_validation_data[
    (small_grid_validation_data['month'] != 1) &
    (small_grid_validation_data['month'] != 2) &
    (small_grid_validation_data['month'] != 11) &
    (small_grid_validation_data['month'] != 12)
    ]
small_grid_validation_data.drop(columns=['hour', 'day_of_week', 'month'], inplace=True)
small_grid_validation_data.head()

small_grid_id,61,615,616,700,701,785,788,789,1626,1627,...,7521,7522,7542,7543,7544,7606,7607,7626,7627,7770
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [398]:
# transform data
voronoi_grid_validation_data = validation_data.resample('3H').voronoi_grid_id.value_counts().unstack().fillna(0)
voronoi_grid_validation_data['hour'] = voronoi_grid_validation_data.index.hour
voronoi_grid_validation_data['day_of_week'] = voronoi_grid_validation_data.index.dayofweek
voronoi_grid_validation_data['month'] = voronoi_grid_validation_data.index.month
voronoi_grid_validation_data.sort_index(inplace=True)
voronoi_grid_validation_data = voronoi_grid_validation_data[
    (voronoi_grid_validation_data['month'] != 1) &
    (voronoi_grid_validation_data['month'] != 2) &
    (voronoi_grid_validation_data['month'] != 11) &
    (voronoi_grid_validation_data['month'] != 12)
    ]
voronoi_grid_validation_data.drop(columns=['hour', 'day_of_week', 'month'], inplace=True)
voronoi_grid_validation_data.head()

voronoi_grid_id,0,1,2,3,4,5,6,7,8,9,...,1139,1140,1141,1142,1144,1145,1146,1147,1148,1149
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [399]:
# Get columns as sets
columns_df1 = set(voronoi_grid_validation_data.columns)
columns_df2 = set(voronoi_grid_poisson_demand.columns)

# Find columns unique to each dataframe
unique_to_df1 = columns_df1 - columns_df2
unique_to_df2 = columns_df2 - columns_df1

# Alternatively, find all columns that exist only in one dataframe
unique_columns = columns_df1.symmetric_difference(columns_df2)

# Results
print("Columns only in val:", unique_to_df1)
print("Columns only in demand:", unique_to_df2)
print("Columns unique to either df1 or df2:", unique_columns)

Columns only in val: {1149, 1044, 813}
Columns only in demand: {1030, 779, 907, 782, 910, 911, 913, 786, 1039, 789, 151, 152, 1049, 1054, 416, 929, 930, 803, 1058, 805, 809, 684, 47, 431, 815, 946, 1071, 948, 1074, 1077, 827, 1083, 451, 836, 963, 838, 968, 843, 973, 976, 851, 852, 981, 1107, 855, 1112, 985, 1115, 994, 483, 743, 872, 879, 880, 1138, 886, 1143, 382, 383}
Columns unique to either df1 or df2: {47, 151, 152, 382, 383, 416, 431, 451, 483, 684, 743, 779, 782, 786, 789, 803, 805, 809, 813, 815, 827, 836, 838, 843, 851, 852, 855, 872, 879, 880, 886, 907, 910, 911, 913, 929, 930, 946, 948, 963, 968, 973, 976, 981, 985, 994, 1030, 1039, 1044, 1049, 1054, 1058, 1071, 1074, 1077, 1083, 1107, 1112, 1115, 1138, 1143, 1149}


In [400]:
def align_dataframes(val_data, demand_data):
    # Get columns as sets
    columns_val = set(val_data.columns)
    columns_demand = set(demand_data.columns)

    # Find columns unique to each dataframe
    unique_to_val = columns_val - columns_demand
    unique_to_demand = columns_demand - columns_val

    # Add missing columns to validation data
    for col in unique_to_demand:
        val_data[col] = 0

    val_data.drop(columns=unique_to_val, inplace=True)

    # Reorder columns
    demand_data = demand_data[val_data.columns]

    return val_data, demand_data

# Baseline
Calculate baseline mean

In [401]:
# load reservation data
training_data = pd.read_pickle(training_data_input_filepath)
training_data['startTime'] = pd.to_datetime(training_data['startTime'])
training_data.set_index('startTime', inplace=True)
training_data.drop(columns=['endTime', 'temperature', 'precipitation', 'startLat', 'startLon', 'endLat', 'endLon'], inplace=True)
training_data.head()

,voronoi_grid_id,small_grid_id,large_grid_id,community_small_grid_id,community_voronoi_grid_id
startTime,,,,,
2023-02-27 14:05:52,263,4717,195,7,0
2021-05-28 13:30:25,863,6600,266,0,4
2022-08-27 17:47:21,468,5484,212,0,7
2021-07-22 16:31:23,287,3963,163,6,9
2018-09-15 16:14:54,335,3269,126,12,4


In [402]:
large_grid_baseline_training_data = training_data.resample('3H').large_grid_id.value_counts().unstack().fillna(0)
large_grid_baseline_training_data['hour'] = large_grid_baseline_training_data.index.hour
large_grid_baseline_training_data['day_of_week'] = large_grid_baseline_training_data.index.dayofweek
large_grid_baseline_training_data['month'] = large_grid_baseline_training_data.index.month
large_grid_baseline_training_data = large_grid_baseline_training_data[
    (large_grid_baseline_training_data['month'] != 1) &
    (large_grid_baseline_training_data['month'] != 2) &
    (large_grid_baseline_training_data['month'] != 11) &
    (large_grid_baseline_training_data['month'] != 12)
    ]
large_grid_baseline_training_data.drop(columns=['hour', 'day_of_week', 'month'], inplace=True)
large_grid_baseline_training_data.head()

large_grid_id,12,21,53,69,70,75,76,90,103,107,...,284,297,298,299,300,301,312,313,318,333
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [403]:
small_grid_baseline_training_data = training_data.resample('3H').small_grid_id.value_counts().unstack().fillna(0)
small_grid_baseline_training_data['hour'] = small_grid_baseline_training_data.index.hour
small_grid_baseline_training_data['day_of_week'] = small_grid_baseline_training_data.index.dayofweek
small_grid_baseline_training_data['month'] = small_grid_baseline_training_data.index.month
small_grid_baseline_training_data = small_grid_baseline_training_data[
    (small_grid_baseline_training_data['month'] != 1) &
    (small_grid_baseline_training_data['month'] != 2) &
    (small_grid_baseline_training_data['month'] != 11) &
    (small_grid_baseline_training_data['month'] != 12)
    ]
small_grid_baseline_training_data.drop(columns=['hour', 'day_of_week', 'month'], inplace=True)
small_grid_baseline_training_data.head()

small_grid_id,61,615,616,700,701,785,786,788,789,1626,...,7606,7607,7626,7627,7628,7712,7769,7770,7854,8127
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [404]:
voronoi_grid_baseline_training_data = training_data.resample('3H').voronoi_grid_id.value_counts().unstack().fillna(0)
voronoi_grid_baseline_training_data['hour'] = voronoi_grid_baseline_training_data.index.hour
voronoi_grid_baseline_training_data['day_of_week'] = voronoi_grid_baseline_training_data.index.dayofweek
voronoi_grid_baseline_training_data['month'] = voronoi_grid_baseline_training_data.index.month
voronoi_grid_baseline_training_data = voronoi_grid_baseline_training_data[
    (voronoi_grid_baseline_training_data['month'] != 1) &
    (voronoi_grid_baseline_training_data['month'] != 2) &
    (voronoi_grid_baseline_training_data['month'] != 11) &
    (voronoi_grid_baseline_training_data['month'] != 12)
    ]
voronoi_grid_baseline_training_data.drop(columns=['hour', 'day_of_week', 'month'], inplace=True)
voronoi_grid_baseline_training_data.head()

voronoi_grid_id,0,1,2,3,4,5,6,7,8,9,...,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [405]:
large_grid_validation_data, large_grid_poisson_demand = align_dataframes(large_grid_validation_data, large_grid_poisson_demand)
large_grid_validation_data, large_grid_negative_binomial_demand = align_dataframes(large_grid_validation_data, large_grid_negative_binomial_demand)

small_grid_validation_data, small_grid_poisson_demand = align_dataframes(small_grid_validation_data, small_grid_poisson_demand)
small_grid_validation_data, small_grid_negative_binomial_demand = align_dataframes(small_grid_validation_data, small_grid_negative_binomial_demand)

voronoi_grid_validation_data, voronoi_grid_poisson_demand = align_dataframes(voronoi_grid_validation_data, voronoi_grid_poisson_demand)
voronoi_grid_validation_data, voronoi_grid_negative_binomial_demand = align_dataframes(voronoi_grid_validation_data, voronoi_grid_negative_binomial_demand)

In [406]:
large_grid_baseline_training_data, large_grid_baseline_validation_data = align_dataframes(large_grid_baseline_training_data, large_grid_validation_data)
small_grid_baseline_training_data, small_grid_baseline_validation_data = align_dataframes(small_grid_baseline_training_data, small_grid_validation_data)
voronoi_grid_baseline_training_data, voronoi_grid_baseline_validation_data = align_dataframes(voronoi_grid_baseline_training_data, voronoi_grid_validation_data)

In [407]:
def baseline_mean(baseline_training_data, baseline_validation_data):
    mean_training_data = baseline_training_data.mean()
    # mean_training_data = baseline_training_data.median()
    # Create a new DataFrame with the same index and columns as baseline_validation_data
    baseline_data = pd.DataFrame(index=baseline_validation_data.index, columns=baseline_validation_data.columns)

    # Fill the DataFrame with the corresponding values from mean_training_data
    for column in baseline_data.columns:
        baseline_data[column] = mean_training_data[column]

    return baseline_data

In [408]:
large_grid_mean_baseline_data = baseline_mean(large_grid_baseline_training_data, large_grid_baseline_validation_data)
small_grid_mean_baseline_data = baseline_mean(small_grid_baseline_training_data, small_grid_baseline_validation_data)
voronoi_grid_mean_baseline_data = baseline_mean(voronoi_grid_baseline_training_data, voronoi_grid_baseline_validation_data)
large_grid_mean_baseline_data.head()

large_grid_id,12,21,53,69,70,75,76,90,103,107,...,284,297,298,299,300,301,312,313,318,333
startTime,,,,,,,,,,,,,,,,,,,,,
2017-10-14 12:00:00,0.000098,0.050964,0.00059,0.000787,0.001771,0.000394,0.026761,0.003935,0.002558,0.023613,...,0.031287,0.010626,0.002066,0.000394,0.013479,0.024006,0.000295,0.002263,0.000492,0.000098
2017-10-14 15:00:00,0.000098,0.050964,0.00059,0.000787,0.001771,0.000394,0.026761,0.003935,0.002558,0.023613,...,0.031287,0.010626,0.002066,0.000394,0.013479,0.024006,0.000295,0.002263,0.000492,0.000098
2017-10-14 18:00:00,0.000098,0.050964,0.00059,0.000787,0.001771,0.000394,0.026761,0.003935,0.002558,0.023613,...,0.031287,0.010626,0.002066,0.000394,0.013479,0.024006,0.000295,0.002263,0.000492,0.000098
2017-10-14 21:00:00,0.000098,0.050964,0.00059,0.000787,0.001771,0.000394,0.026761,0.003935,0.002558,0.023613,...,0.031287,0.010626,0.002066,0.000394,0.013479,0.024006,0.000295,0.002263,0.000492,0.000098
2017-10-15 06:00:00,0.000098,0.050964,0.00059,0.000787,0.001771,0.000394,0.026761,0.003935,0.002558,0.023613,...,0.031287,0.010626,0.002066,0.000394,0.013479,0.024006,0.000295,0.002263,0.000492,0.000098


## Calculate metrics

In [409]:
def normalize_array_columns(arr):
    # Normalize each column so that it sums to 1, avoiding division by zero
    column_sums = arr.sum(axis=0, keepdims=True)
    column_sums[column_sums == 0] = 1  
    return arr / column_sums

In [410]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    kl_1 = kl_div(normalize_array_columns(y_true), normalize_array_columns(y_pred))
    kl_2 = kl_div(normalize_array_columns(y_pred), normalize_array_columns(y_true))
    return [mse, mae, r2, np.median(kl_1), np.median(kl_2)]

In [411]:
# large grid poisson
large_grid_poisson_metrics = calculate_metrics(large_grid_validation_data.values, large_grid_poisson_demand.values)
large_grid_poisson_metrics

[0.12157543684767146,
 0.15862404634618943,
 -0.43962257687346273,
 6.630992898432523e-05,
 inf]

In [412]:
# large grid negative binomial
large_grid_negative_binomial_metrics = calculate_metrics(large_grid_validation_data.values, large_grid_negative_binomial_demand.values)
large_grid_negative_binomial_metrics

[0.12523328150916732,
 0.16094148295380498,
 -0.47150605449393324,
 6.572338171197355e-05,
 inf]

In [413]:
# small grid poisson
small_grid_poisson_metrics = calculate_metrics(small_grid_validation_data.values, small_grid_poisson_demand.values)
small_grid_poisson_metrics

[0.004776814958463866,
 0.004947332630565482,
 -0.0011370730337188202,
 8.394438479731591e-05,
 inf]

In [414]:
# small grid negative binomial
small_grid_negative_binomial_metrics = calculate_metrics(small_grid_validation_data.values, small_grid_negative_binomial_demand.values)
small_grid_negative_binomial_metrics

[0.004785106402226335,
 0.0049526703941454465,
 -0.0002044254603581663,
 8.390308387207992e-05,
 inf]

In [415]:
# voronoi grid poisson
voronoi_grid_poisson_metrics = calculate_metrics(voronoi_grid_validation_data.values, voronoi_grid_poisson_demand.values)
voronoi_grid_poisson_metrics

[0.004403104349904976,
 0.00478007186126415,
 -0.0562003944887186,
 9.241385362135727e-05,
 inf]

In [416]:
# voronoi grid negative binomial
voronoi_grid_negative_binomial_metrics = calculate_metrics(voronoi_grid_validation_data.values, voronoi_grid_negative_binomial_demand.values)
voronoi_grid_negative_binomial_metrics

[0.004637465486383538,
 0.00496586827420333,
 -0.08956295139311075,
 9.0762233881802e-05,
 inf]

### Calculate Baseline Metrics

In [417]:
large_grid_mean_baseline_metrics = calculate_metrics(large_grid_baseline_validation_data.values, large_grid_mean_baseline_data.values)
small_grid_mean_baseline_metrics = calculate_metrics(small_grid_baseline_validation_data.values, small_grid_mean_baseline_data.values)
voronoi_grid_mean_baseline_metrics = calculate_metrics(voronoi_grid_baseline_validation_data.values, voronoi_grid_mean_baseline_data.values)

## Save metrics

In [418]:
# create dataframe with metrics
model_evaluation_metrics = pd.DataFrame({'model': ['large_grid_poisson', 'large_grid_negative_binomial', 'small_grid_poisson', 'small_grid_negative_binomial', 'voronoi_grid_poisson', 'voronoi_grid_negative_binomial'], 'mse': [large_grid_poisson_metrics[0], large_grid_negative_binomial_metrics[0], small_grid_poisson_metrics[0], small_grid_negative_binomial_metrics[0], voronoi_grid_poisson_metrics[0], voronoi_grid_negative_binomial_metrics[0]], 'mae': [large_grid_poisson_metrics[1], large_grid_negative_binomial_metrics[1], small_grid_poisson_metrics[1], small_grid_negative_binomial_metrics[1], voronoi_grid_poisson_metrics[1], voronoi_grid_negative_binomial_metrics[1]], 'r2': [large_grid_poisson_metrics[2], large_grid_negative_binomial_metrics[2], small_grid_poisson_metrics[2], small_grid_negative_binomial_metrics[2], voronoi_grid_poisson_metrics[2], voronoi_grid_negative_binomial_metrics[2]], 'kl_1': [large_grid_poisson_metrics[3], large_grid_negative_binomial_metrics[3], small_grid_poisson_metrics[3], small_grid_negative_binomial_metrics[3], voronoi_grid_poisson_metrics[3], voronoi_grid_negative_binomial_metrics[3]], 'kl_2': [large_grid_poisson_metrics[4], large_grid_negative_binomial_metrics[4], small_grid_poisson_metrics[4], small_grid_negative_binomial_metrics[4], voronoi_grid_poisson_metrics[4], voronoi_grid_negative_binomial_metrics[4]]})
model_evaluation_metrics

,model,mse,mae,r2,kl_1,kl_2
0,large_grid_poisson,0.121575,0.158624,-0.439623,0.000066,inf
1,large_grid_negative_binomial,0.125233,0.160941,-0.471506,0.000066,inf
2,small_grid_poisson,0.004777,0.004947,-0.001137,0.000084,inf
3,small_grid_negative_binomial,0.004785,0.004953,-0.000204,0.000084,inf
4,voronoi_grid_poisson,0.004403,0.004780,-0.056200,0.000092,inf
5,voronoi_grid_negative_binomial,0.004637,0.004966,-0.089563,0.000091,inf


In [419]:
# save metrics
model_evaluation_metrics.to_pickle(ml_evaluation_output_path)

In [420]:
# create dataframe with baseline metrics
baseline_metrics = pd.DataFrame({'model': ['baseline_large_grid_mean', 'baseline_small_grid_mean', 'baseline_voronoi_grid_mean'], 'mse': [large_grid_mean_baseline_metrics[0], small_grid_mean_baseline_metrics[0], voronoi_grid_mean_baseline_metrics[0]], 'mae': [large_grid_mean_baseline_metrics[1], small_grid_mean_baseline_metrics[1], voronoi_grid_mean_baseline_metrics[1]], 'r2': [large_grid_mean_baseline_metrics[2], small_grid_mean_baseline_metrics[2], voronoi_grid_mean_baseline_metrics[2]], 'kl_1': [large_grid_mean_baseline_metrics[3], small_grid_mean_baseline_metrics[3], voronoi_grid_mean_baseline_metrics[3]], 'kl_2': [large_grid_mean_baseline_metrics[4], small_grid_mean_baseline_metrics[4], voronoi_grid_mean_baseline_metrics[4]]})
baseline_metrics

,model,mse,mae,r2,kl_1,kl_2
0,baseline_large_grid_mean,0.088369,0.145355,-0.160422,0.000118,inf
1,baseline_small_grid_mean,0.004886,0.017472,-0.016813,0.000118,inf
2,baseline_voronoi_grid_mean,0.004207,0.015348,-0.014951,0.000118,inf


In [421]:
# load dl metrics
dl_metrics = pd.read_pickle(dl_evaluation_output_path)
dl_metrics

,model,mse,mae,r2,kl_1,kl_2
0,lstm,0.028057,0.059605,-10.580745,0.000072,inf
1,lstm_weather,0.033568,0.064695,-24.384932,0.000071,inf
2,lstm_cc,0.000951,0.002973,-0.043849,0.000071,inf
3,lstm_cc_weather,0.001042,0.005058,-0.168338,0.000070,inf
4,lstm_voronoi_cc,0.000722,0.002496,-0.000699,0.000027,inf
5,lstm_voronoi_cc_weather,0.000790,0.004218,-0.000699,0.000018,inf


In [422]:
# combine metrics
combined_metrics = pd.concat([model_evaluation_metrics, dl_metrics, baseline_metrics])
combined_metrics

,model,mse,mae,r2,kl_1,kl_2
0,large_grid_poisson,0.121575,0.158624,-0.439623,0.000066,inf
1,large_grid_negative_binomial,0.125233,0.160941,-0.471506,0.000066,inf
2,small_grid_poisson,0.004777,0.004947,-0.001137,0.000084,inf
3,small_grid_negative_binomial,0.004785,0.004953,-0.000204,0.000084,inf
4,voronoi_grid_poisson,0.004403,0.004780,-0.056200,0.000092,inf
5,voronoi_grid_negative_binomial,0.004637,0.004966,-0.089563,0.000091,inf
0,lstm,0.028057,0.059605,-10.580745,0.000072,inf
1,lstm_weather,0.033568,0.064695,-24.384932,0.000071,inf
2,lstm_cc,0.000951,0.002973,-0.043849,0.000071,inf
3,lstm_cc_weather,0.001042,0.005058,-0.168338,0.000070,inf


In [423]:
# save combined metrics
combined_metrics.to_pickle(evaluation_output_path)